<a href="https://colab.research.google.com/github/Engineer-D/My-Practices/blob/main/Semantic_Search_V0_SBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 72.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=6d8d278925a5c87e74802a1525d06fdd74e2ddd71af07d86c64d8a857f6444b0
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [7]:
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 66.2 MB/s eta 0:00:00


In [1]:
import pandas as pd
df = pd.read_csv("/content/QuestionTest.csv", delimiter="\t", header = None)

In [2]:
df.columns = ["Question", "Answer"]
df['embeddings'] = 0
df['meta'] = 0

In [3]:
df.head()

,Question,Answer,embeddings,meta
0,Do you have a member of your organization with...,Yes,0,0
1,Is a background check required for all employe...,No,0,0
2,Does the organization have written information...,No,0,0
3,"If yes, please provide copies when respondi...",See Comments,0,0
4,Does the organization have a written password ...,Yes,0,0


In [4]:
from tqdm.auto import tqdm

In [6]:
from sentence_transformers import SentenceTransformer

# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')

# create sentence embeddings
sentence_embeddings = model.encode(df['Question'])
sentence_embeddings.shape

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

(91, 768)

In [8]:
import faiss
d = sentence_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.is_trained

True

In [9]:
index.add(sentence_embeddings)
index.ntotal

91

In [47]:
query = list(df['Question'][[2,34,52,12,77,3]])

In [12]:
k = 3
xq = model.encode(query)

In [13]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[ 2 69 14]
 [34 14 28]
 [52  7 59]
 [12 61 10]
 [77  1 61]
 [ 3 15 21]]
CPU times: user 3.77 ms, sys: 0 ns, total: 3.77 ms
Wall time: 7.13 ms


In [43]:
for i in tqdm(range(len(I))):
  print(f"Question: {query[i]}")
  for similar_question in I[i]:
    print(f"Similar Question: {df['Question'].iloc[similar_question]}")
    answer = df[df["Question"] == df['Question'].iloc[similar_question]]["Answer"][similar_question]
    print(f'Answer: {answer}')
  print()

  0%|          | 0/6 [00:00<?, ?it/s]

Question: Does the organization have written information security policies?
Similar Question: Does the organization have written information security policies?
Answer: No
Similar Question: Does your organization  have an information asset management policy in place?
Answer: Yes
Similar Question: Does the organization have security measures in place for data protection?
Answer: No

Question: What security monitoring procedures are in place?
Similar Question: What security monitoring procedures are in place?
Answer: See Comments
Similar Question: Does the organization have security measures in place for data protection?
Answer: No
Similar Question: Are Intrusion Detection Systems (IDS) or Intrusion Prevention Systems (IPS) used by your organization?
Answer: Yes

Question: Does the organization have a formal Incident Response plan?
Similar Question: Does the organization have a formal Incident Response plan?
Answer: Yes
Similar Question: Does the organization have a formal change control 

In [48]:
r_query = ['Is there any information security policy at your company?',
 'How do you handle security monitoring?',
 'As regards formal Incident response plans do you have any?',
 'Before deploying to production environment do test for security and security patches?',
 'Is there any background check on new hires or potentials',
 'provide copies when responding to this accreditation']

In [49]:
k = 3
xq = model.encode(r_query)

In [50]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[69  0 14]
 [34 14 28]
 [52  7 79]
 [12 10  9]
 [61 77 90]
 [ 3 58 33]]
CPU times: user 1.53 ms, sys: 60 µs, total: 1.59 ms
Wall time: 1.28 ms


In [51]:
for i in tqdm(range(len(I))):
  print(f"Question: {query[i]}")
  for similar_question in I[i]:
    print(f"Similar Question: {df['Question'].iloc[similar_question]}")
    answer = df[df["Question"] == df['Question'].iloc[similar_question]]["Answer"][similar_question]
    print(f'Answer: {answer}')
  print()

  0%|          | 0/6 [00:00<?, ?it/s]

Question: Does the organization have written information security policies?
Similar Question: Does your organization  have an information asset management policy in place?
Answer: Yes
Similar Question: Do you have a member of your organization with dedicated information security function?
Answer: Yes
Similar Question: Does the organization have security measures in place for data protection?
Answer: No

Question: What security monitoring procedures are in place?
Similar Question: What security monitoring procedures are in place?
Answer: See Comments
Similar Question: Does the organization have security measures in place for data protection?
Answer: No
Similar Question: Are Intrusion Detection Systems (IDS) or Intrusion Prevention Systems (IPS) used by your organization?
Answer: Yes

Question: Does the organization have a formal Incident Response plan?
Similar Question: Does the organization have a formal Incident Response plan?
Answer: Yes
Similar Question: Does the organization have a